## Willow Flycatcher Migration

<img src="willow-flycatcher.png" alt="Willow Flycatcher" width="720" height="550" longdesc="https://macaulaylibrary.org/asset/451259001" /> 

**About**

Willow Flycatchers (*Empidonax traillii*) specialize in areas with willows and other shrubs [near running and still water](https://www.audubon.org/field-guide/bird/willow-flycatcher). They are about 6 inches in length with brown, gray, and white feathers, a rounded wing, and a square-tipped tail. The call of a Willow Flycatcher is a chirp, buzz, or trill and matches an undulating pattern. Nests are placed 4-15 feet above water or damp ground and constructed as an open cup of grass, bark, and plant fibers. The species migrates long distances, breeding in the U.S. and Canada and wintering in Mexico, Central America, and northern South America. They are common in most locations in their range despite a [25% decline](https://www.allaboutbirds.org/guide/Willow_Flycatcher/lifehistory) in population between 1966 to 2019. The loss of wet marshes, wet meadows, and riparian vegetation has contributed to [declining species abundance](https://www.fs.usda.gov/detail/tahoe/landmanagement/resourcemanagement/?cid=stelprdb5357314#:~:text=The%20scientific%20name%20for%20willow,t). According to the Bird Genoscape Project, there are seven geneticially [distinct populations](https://www.birdgenoscape.org/willow-flycatcher/) of Willow Flycatcher in North America: the Pacific Northwest, Kern, California, southern California, White Mountain, Arizona, Interior West, Southwest, and Eastern. In my home state of California, there are three [endangered subspecies](https://www.fs.usda.gov/detail/tahoe/landmanagement/resourcemanagement/?cid=stelprdb5357314#:~:text=The%20scientific%20name%20for%20willow,t): Southwestern Willow Flycatcher in central and southern California (Federal and State), Little Willow Flycatcher in high elevation Sierra Nevada (State), and Great Basin Willow Flycatcher in desert riparian area (State). Researchers have found that the Southwestern Willow Flycatcher has a higher prevalence of gene variants today compared to 100 years ago that are associated with [adapting to wet and humid conditions](https://www.allaboutbirds.org/news/endangered-willow-flycatchers-in-san-diego-are-adapting-to-climate-change/). This difference is likely due to interbreeding with species in the Southwest and Pacific Northwest, producing an evolutionary response to climate change. Adaptations like these are why it is vital to preserve the interconnectivity of species populations through the protection of habitat and landscape mobility.

##### Imports

In [1]:
%%bash
pip install pygbif
pip install geoviews

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import pathlib
import time
import zipfile
from getpass import getpass
from glob import glob
import pandas as pd
import geopandas as gpd

import pygbif.occurrences as occ
import pygbif.species as species

# get month names
import calendar

# libraries for Dynamic mapping
import geoviews as gv
import hvplot.pandas
import cartopy.crs as ccrs
import panel as pn
pn.extension()

#### Analysis

In [3]:
# Create data directory in the home folder
data_dir = os.path.join(
    # Home directory
    pathlib.Path.home(),
    # Earth analytics data directory
    'earth-analytics',
    'data',
    # Project directory
    'species-distribution',
)
os.makedirs(data_dir, exist_ok=True)

# Define the directory name for GBIF data
gbif_dir = os.path.join(data_dir, 'willow-flycatcher', '2023')

In [4]:
gbif_dir

'/home/jovyan/earth-analytics/data/species-distribution/willow-flycatcher/2023'

##### Access GBIF

In [5]:
reset_credentials = False
# GBIF needs a username, password, and email
credentials = dict(
    GBIF_USER=(input, ''),
    GBIF_PWD=(getpass, ''),
    GBIF_EMAIL=(input, ''),
)

for env_variable, (prompt_func, prompt_text) in credentials.items():
    # Delete credential from environment if requested
    if reset_credentials and (env_variable in os.environ):
        os.environ.pop(env_variable)
    # Ask for credential and save to environment
    if not env_variable in os.environ:
        os.environ[env_variable] = prompt_func(prompt_text)


In [6]:
# Query species
species_info = species.name_lookup('Empidonax traillii', rank='SPECIES')

# Get the first result
first_result = species_info['results'][0]

# Get the species key (nubKey)
species_key = first_result['nubKey']

# Check the result
first_result['species'], species_key

('Empidonax traillii', 2482786)

##### Download data from GBIF

In [7]:
# Only download once
gbif_pattern = os.path.join(gbif_dir, '*.csv')
if not glob(gbif_pattern):
    # Submit query to GBIF
    gbif_query = occ.download([
        "speciesKey = 2482786",
        "year = 2023",
        "hasCoordinate = TRUE",
    ],
    user=credentials['GBIF_USER'][1], 
    pwd=credentials['GBIF_PWD'][1], 
    email=credentials['GBIF_EMAIL'][1])

    # Only download once
    if not 'GBIF_DOWNLOAD_KEY' in os.environ:
        os.environ['GBIF_DOWNLOAD_KEY'] = gbif_query[0]

        # Wait for the download to build
        wait = occ.download_meta(os.environ['GBIF_DOWNLOAD_KEY'])['status'] 
        while not wait=='SUCCEEDED':
            wait = occ.download_meta(os.environ['GBIF_DOWNLOAD_KEY'])['status'] 
            time.sleep(5)

        # Download GBIF data
        download_info = occ.download_get(
            os.environ['GBIF_DOWNLOAD_KEY'], 
            path=data_dir)

        # Unzip GBIF data
        with zipfile.ZipFile(download_info['path']) as download_zip:
            download_zip.extractall(path=gbif_dir)

# Find the extracted .csv file path
gbif_path = glob(gbif_pattern)[0]

In [8]:
gbif_path

'/home/jovyan/earth-analytics/data/species-distribution/willow-flycatcher/2023/0001229-241007104925546.csv'

##### Load the GBIF data into Python

In [9]:
!head -n 2 $gbif_path 

gbifID	datasetKey	occurrenceID	kingdom	phylum	class	order	family	genus	species	infraspecificEpithet	taxonRank	scientificName	verbatimScientificName	verbatimScientificNameAuthorship	countryCode	locality	stateProvince	occurrenceStatus	individualCount	publishingOrgKey	decimalLatitude	decimalLongitude	coordinateUncertaintyInMeters	coordinatePrecision	elevation	elevationAccuracy	depth	depthAccuracy	eventDate	day	month	year	taxonKey	speciesKey	basisOfRecord	institutionCode	collectionCode	catalogNumber	recordNumber	identifiedBy	dateIdentified	license	rightsHolder	recordedBy	typeStatus	establishmentMeans	lastInterpreted	mediaType	issue
4846727844	50c9509d-22c7-4a22-a47d-8c48425ef4a7	https://www.inaturalist.org/observations/176130744	Animalia	Chordata	Aves	Passeriformes	Tyrannidae	Empidonax	Empidonax traillii		SPECIES	Empidonax traillii (Audubon, 1828)	Empidonax traillii		US		Illinois	PRESENT		28eb1a3f-1c15-4a95-931a-4af90ecb574d	41.840597	-88.075757	61.0						2023-08-01T09:28	1	8	2023	2482786	

##### Year 2023

In [10]:
# Load the GBIF data
gbif_df_2023 = pd.read_csv(
    gbif_path, 
    delimiter='\t',
    index_col='gbifID',
    on_bad_lines='skip',
    usecols=['gbifID', 'month', 'year', 'countryCode', 'stateProvince', 'decimalLatitude', 'decimalLongitude']
)

gbif_df_2023.head(2)

,countryCode,stateProvince,decimalLatitude,decimalLongitude,month,year
gbifID,,,,,,
4846727844,US,Illinois,41.840597,-88.075757,8,2023
4458666436,CA,Alberta,51.170300,-115.593645,6,2023


##### Breeding locations (high to low latitude)

In [11]:
# Canada

wf_CA = gbif_df_2023.loc[gbif_df_2023['countryCode'] == 'CA']
wf_CA.value_counts()

countryCode  stateProvince     decimalLatitude  decimalLongitude  month  year
CA           Ontario           43.628270        -79.32917         5      2023    160
             British Columbia  49.234290        -122.79964        7      2023    142
                               48.319800        -123.54715        8      2023    139
                               49.234290        -122.79964        6      2023    115
             Ontario           41.955400        -82.51400         5      2023    109
                                                                                ... 
                               42.325730        -82.89813         6      2023      1
             British Columbia  49.235115        -122.89093        6      2023      1
             Ontario           42.325302        -82.92369         5      2023      1
             British Columbia  49.235120        -122.63271        6      2023      1
             Ontario           42.335026        -81.85847         7     

In [12]:
# United States 

wf_US = gbif_df_2023.loc[gbif_df_2023['countryCode'] == 'US']
wf_US.value_counts()

countryCode  stateProvince  decimalLatitude  decimalLongitude  month  year
US           Pennsylvania   39.889350        -75.260150        5      2023    227
                                                               6      2023    216
             Illinois       41.963383        -87.634420        5      2023    177
             Ohio           41.451912        -82.667366        5      2023    169
             Pennsylvania   40.271667        -76.247734        7      2023    166
                                                                             ... 
             Wyoming        44.811672        -108.800170       5      2023      1
                            44.659695        -106.948850       5      2023      1
                            44.510040        -109.146800       6      2023      1
                            44.462100        -110.853570       5      2023      1
                            44.438520        -109.217010       8      2023      1
Name: count, Length: 32

##### Wintering locations (high to low latitude)

In [13]:
# Mexico

wf_MX = gbif_df_2023.loc[gbif_df_2023['countryCode'] == 'MX']
wf_MX.value_counts()

countryCode  stateProvince  decimalLatitude  decimalLongitude  month  year
MX           Tabasco        17.989754        -92.973090        9      2023    18
             Oaxaca         15.761899        -96.127110        11     2023    12
             Sonora         27.009628        -108.913380       10     2023    10
             Veracruz       19.469866        -96.788970        9      2023     8
             Nayarit        21.528181        -105.219970       2      2023     7
                                                                              ..
             Veracruz       18.543131        -95.148419        5      2023     1
                            18.455866        -95.185870        10     2023     1
                            19.110413        -96.121056        5      2023     1
                            19.067734        -96.075860        5      2023     1
                            19.490616        -96.332400        5      2023     1
Name: count, Length: 212, dtype: i

In [14]:
# Nicaragua

wf_NI = gbif_df_2023.loc[gbif_df_2023['countryCode'] == 'NI']
wf_NI.value_counts()

countryCode  stateProvince  decimalLatitude  decimalLongitude  month  year
NI           Masaya         12.087877        -85.989650        3      2023    10
                            12.090204        -85.989850        1      2023     8
             Chinandega     12.957290        -87.496890        12     2023     7
                            12.889106        -87.505250        11     2023     4
                            12.888454        -87.504590        12     2023     4
             Masaya         12.089507        -85.988840        2      2023     4
             Managua        12.239460        -86.415405        12     2023     3
             Chinandega     12.889481        -87.504870        1      2023     3
             Masaya         12.087877        -85.989650        12     2023     3
                            11.950830        -86.239630        1      2023     3
             Managua        12.239460        -86.415405        11     2023     3
             Chinandega     12.888

In [15]:
# Colombia

wf_CO = gbif_df_2023.loc[gbif_df_2023['countryCode'] == 'CO']
wf_CO.value_counts()

countryCode  stateProvince  decimalLatitude  decimalLongitude  month  year
CO           Boyacá         6.047542         -74.261986        11     2023    13
                                                               4      2023    10
             Magdalena      11.033774        -74.208290        4      2023    10
             Guaviare       2.583213         -72.662506        1      2023     9
             Boyacá         6.047542         -74.261986        10     2023     8
                                                                              ..
             Tolima         4.706385         -74.903110        10     2023     1
                            4.424295         -75.172520        4      2023     1
             Santander      7.222714         -73.119780        11     2023     1
             Tolima         4.451201         -75.221540        2      2023     1
                            4.447423         -75.225690        3      2023     1
Name: count, Length: 137, dtype: i

##### Convert the GBIF data to a GeoDataFrame

In [16]:
"""
Coordinate reference system:
EPSG:4326, also known as the WGS84 projection, is a coordinate system that represents the Earth as a 3D ellipsoid. 
It's used in Google Earth, GPS systems, and by organizations that provide GIS data for the entire globe or many countries.
"""

gdf_monthly = (
    gpd.GeoDataFrame(
        gbif_df_2023, 
        geometry=gpd.points_from_xy(
            gbif_df_2023.decimalLongitude, 
            gbif_df_2023.decimalLatitude), 
        crs="EPSG:4326")
    # Select the desired columns
    [['month', 'geometry']]
)

gdf_monthly

,month,geometry
gbifID,,
4846727844,8,POINT (-88.07576 41.8406)
4458666436,6,POINT (-115.59364 51.1703)
4171308981,7,POINT (-97.08144 43.17027)
4527914925,7,POINT (-115.79344 49.49855)
4694654654,7,POINT (-123.03056 43.7905)
...,...,...
4771133627,7,POINT (-119.28043 50.70975)
4681902738,5,POINT (-87.15982 38.9803)
4829948692,6,POINT (-121.91132 47.59308)


In [17]:
gdf_canada = (
    gpd.GeoDataFrame(
        wf_CA, 
        geometry=gpd.points_from_xy(
            wf_CA.decimalLongitude, 
            wf_CA.decimalLatitude), 
        crs="EPSG:4326")
    # Select the desired columns
    [['month', 'stateProvince', 'geometry']]
)

gdf_canada.reset_index(inplace=True, drop=True)

gdf_canada

,month,stateProvince,geometry
0,6,Alberta,POINT (-115.59364 51.1703)
1,7,British Columbia,POINT (-115.79344 49.49855)
2,8,British Columbia,POINT (-123.96848 49.71062)
3,6,Ontario,POINT (-80.23159 43.4964)
4,6,British Columbia,POINT (-119.53582 49.30306)
...,...,...,...
18152,6,British Columbia,POINT (-123.00034 49.30584)
18153,7,British Columbia,POINT (-124.7245 49.45994)
18154,6,British Columbia,POINT (-119.27582 50.38022)
18155,6,British Columbia,POINT (-122.61517 49.34824)


##### Download and save ecoregion boundaries

Ecoregions represent boundaries formed by biotic and abiotic conditions: geology, landforms, soils, vegetation, land use, wildlife, climate, and hydrology.

In [18]:
# Set up the ecoregion boundary URL
ecoregions_url = "https://storage.googleapis.com/teow2016/Ecoregions2017.zip"

# Set up a path to save the data on your machine
ecoregions_dir = os.path.join(data_dir, 'wwf_ecoregions')

# Make the ecoregions directory
os.makedirs(ecoregions_dir, exist_ok=True)

# Join ecoregions shapefile path
ecoregions_path = os.path.join(ecoregions_dir, 'wwf_ecoregions.shp')

# Only download once
if not os.path.exists(ecoregions_path):
    ecoregions_gdf = gpd.read_file(ecoregions_url)
    ecoregions_gdf.to_file(ecoregions_path)

In [19]:
%%bash
find ~/earth-analytics/data/species-distribution -name '*.shp'

/home/jovyan/earth-analytics/data/species-distribution/wwf_ecoregions/wwf_ecoregions.shp


In [20]:
# Open up the ecoregions boundaries
ecoregions_gdf = (
    gpd.read_file(ecoregions_path)
    .rename(columns={
        'ECO_NAME': 'name',
        'SHAPE_AREA': 'area'})
    [['name', 'area', 'geometry']]
)

# Name the index so it will match the other data later on
ecoregions_gdf.index.name = 'ecoregion'

# Plot the ecoregions to check download
ecoregions_gdf.plot(edgecolor='black', color='skyblue')

: 

In [2]:
ecoregions_gdf

NameError: name 'ecoregions_gdf' is not defined

##### Identify the ecoregion for each observation

In [23]:
gbif_ecoregion_gdf = (
    ecoregions_gdf
    # Match the coordinate reference system of the GBIF data and the ecoregions
    # transform geometries to a new coordinate reference system
    .to_crs(gdf_monthly.crs)
    # Find ecoregion for each observation
    # spatial join
    .sjoin(
        gdf_monthly,
        how='inner', 
        predicate='contains')
    # Select the required columns
    [['month', 'name']]
)
gbif_ecoregion_gdf

,month,name
ecoregion,,
12,6,Alberta-British Columbia foothills forests
12,6,Alberta-British Columbia foothills forests
12,6,Alberta-British Columbia foothills forests
12,6,Alberta-British Columbia foothills forests
12,6,Alberta-British Columbia foothills forests
...,...,...
833,6,Northern Rockies conifer forests
833,6,Northern Rockies conifer forests
833,6,Northern Rockies conifer forests


##### Count the observations in each ecoregion each month

In [56]:
def get_monthly_regional_observations(df, region_type, occurrence_name):

    occurrence_df = (
        df
        # For each region, for each month...
        .groupby([region_type, 'month'])
        # count the number of occurrences
        .agg(occurrences=(occurrence_name, 'count'))
    )

    # Get rid of rare observations (possible misidentification)
    occurrence_df = occurrence_df[occurrence_df["occurrences"] > 1]

    # Take the mean by region
    mean_occurrences_by_region = (
        occurrence_df
        .groupby([region_type])
        .mean()
    )

    # Take the mean by month
    mean_occurrences_by_month = (
        occurrence_df
        .groupby(['month'])
        .mean()
    )

    # Normalize by space and time for sampling effort
    # This accounts for the number of active observers in each location and time of year
    occurrence_df['norm_occurrences'] = (
        occurrence_df
        / mean_occurrences_by_region
        / mean_occurrences_by_month
    )

    return occurrence_df

In [67]:
occurrence_df = get_monthly_regional_observations(gbif_ecoregion_gdf, 'ecoregion', 'name')

occurrence_df

occurrences  norm_occurrences
ecoregion month                               
12        5               12          0.001558
          6               46          0.003290
          7               10          0.001355
16        5              510          0.004681
          6              718          0.003631
...                      ...               ...
833       5              152          0.000802
          6             1649          0.004795
          7              746          0.004108
          8              220          0.002813
          9               21          0.000645

[537 rows x 2 columns]

##### Breeding observations by month

In [89]:
# Canada

ca_occurrence_df = get_monthly_regional_observations(gdf_canada, 'stateProvince', 'stateProvince')

# for plot
ca_norm_gdf = gdf_canada.join(ca_occurrence_df, on=['stateProvince', 'month'])

ca_norm_gdf

,month,stateProvince,geometry,occurrences,norm_occurrences
0,6,Alberta,POINT (-115.59364 51.1703),334.0,0.003560
1,7,British Columbia,POINT (-115.79344 49.49855),2314.0,0.002493
2,8,British Columbia,POINT (-123.96848 49.71062),1626.0,0.001841
3,6,Ontario,POINT (-80.23159 43.4964),2279.0,0.002133
4,6,British Columbia,POINT (-119.53582 49.30306),4441.0,0.002580
...,...,...,...,...,...
18152,6,British Columbia,POINT (-123.00034 49.30584),4441.0,0.002580
18153,7,British Columbia,POINT (-124.7245 49.45994),2314.0,0.002493
18154,6,British Columbia,POINT (-119.27582 50.38022),4441.0,0.002580
18155,6,British Columbia,POINT (-122.61517 49.34824),4441.0,0.002580


In [2]:
ca_first_row = ca_norm_gdf.head(1)
ca_first_row.month

NameError: name 'ca_norm_gdf' is not defined

In [92]:
ca_norm_gdf.month.unique()

array([ 6,  7,  8,  5,  9, 10])

In [1]:
# account for months without observation in plot
def fill_map(df, empty_months, stateProvince, geometry): 
    month_rows = []

    for month in empty_months:
        empty_month = {'month': month, 'stateProvince': stateProvince, 'geometry': geometry, 'occurrences': 0.0, 'norm_occurrences': 0.0}
        month_rows.append(empty_month)

    df = df.append(month_rows, ignore_index=True)

    return df

In [ ]:
ca_norm_gdf = fill_map(ca_norm_gdf, [0, 1, 2, 3, 4, 11, 12], 'Alberta', geometry) 

ca_norm_gdf.tail(20)

In [90]:
# Speed up processing
ca_norm_gdf.geometry = ca_norm_gdf.simplify(
    .1, preserve_topology=False)

# Change the CRS to Mercator for mapping
ca_norm_gdf = ca_norm_gdf.to_crs(ccrs.Mercator())

ca_norm_gdf

,month,stateProvince,geometry,occurrences,norm_occurrences
0,6,Alberta,POINT (-12867825.7 6618166.089),334.0,0.003560
1,7,British Columbia,POINT (-12890066.667 6327373.526),2314.0,0.002493
2,8,British Columbia,POINT (-13800107.511 6363698.667),1626.0,0.001841
3,6,Ontario,POINT (-8931339.744 5358415.729),2279.0,0.002133
4,6,British Columbia,POINT (-13306666.614 6294027.984),4441.0,0.002580
...,...,...,...,...,...
18152,6,British Columbia,POINT (-13692335.216 6294501.755),4441.0,0.002580
18153,7,British Columbia,POINT (-13884267.273 6320776.588),2314.0,0.002493
18154,6,British Columbia,POINT (-13277723.546 6479452.469),4441.0,0.002580
18155,6,British Columbia,POINT (-13649457.843 6301723.317),4441.0,0.002580


In [91]:
# Get the plot bounds so they don't change with the slider
xmin, ymin, xmax, ymax = ca_norm_gdf.total_bounds

# Define the slider widget
slider = pn.widgets.DiscreteSlider(
    name='month', 
    options={calendar.month_name[i]: i for i in range(1, 13)}
)

# Plot occurrence by ecoregion and month
ca_migration_plot = (
    ca_norm_gdf
    .hvplot(
        c='norm_occurrences',
        groupby='month',
        # Use background tiles
        geo=True, crs=ccrs.Mercator(), tiles='CartoLight',
        title="Willow Flycatcher Migration (Canada)",
        xlim=(xmin, xmax), ylim=(ymin, ymax),
        frame_height=600,
        colorbar=False,
        widgets={'month': slider},
        widget_location='bottom',
        width=500,
        height=500
    )
)

# Show the plot
ca_migration_plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'5b12ea33-9f0f-4f39-b51d-720987086307': {'version…

##### Create a simplified GeoDataFrame for plot

In [57]:
"""
Streamlining plotting with hvplot by simplifying the geometry, projecting it to a Mercator projection that is compatible with
geoviews, and cropping off areas in the Arctic.
"""

# Speed up processing
ecoregions_gdf.geometry = ecoregions_gdf.simplify(
    .1, preserve_topology=False)

# Change the CRS to Mercator for mapping
ecoregions_gdf = ecoregions_gdf.to_crs(ccrs.Mercator())

ecoregions_gdf

,name,area,geometry
ecoregion,,,
0,Adelie Land tundra,0.038948,MULTIPOLYGON EMPTY
1,Admiralty Islands lowland rain forests,0.170599,"POLYGON ((16411777.375 -229101.376, 16384825.7..."
2,Aegean and Western Turkey sclerophyllous and m...,13.844952,"MULTIPOLYGON (((3391149.749 4336064.109, 33846..."
3,Afghan Mountains semi-desert,1.355536,"MULTIPOLYGON (((7369001.698 4093509.259, 73168..."
4,Ahklun and Kilbuck Upland Tundra,8.196573,"MULTIPOLYGON (((-17930832.005 8046779.358, -17..."
...,...,...,...
842,Sulawesi lowland rain forests,9.422097,"MULTIPOLYGON (((14113374.546 501721.962, 14128..."
843,East African montane forests,5.010930,"MULTIPOLYGON (((4298787.669 -137583.786, 42727..."
844,Eastern Arc forests,0.890325,"MULTIPOLYGON (((4267432.68 -493759.165, 428533..."


##### Mapping monthly migration

In [65]:
# Join the occurrences with the plotting GeoDataFrame
occurrence_gdf = ecoregions_gdf.join(occurrence_df)

# Get the plot bounds so they don't change with the slider
xmin, ymin, xmax, ymax = occurrence_gdf.total_bounds

# Define the slider widget
slider = pn.widgets.DiscreteSlider(
    name='month', 
    options={calendar.month_name[i]: i for i in range(1, 13)}
)

# Plot occurrence by ecoregion and month
migration_plot = (
    occurrence_gdf
    .hvplot(
        c='norm_occurrences',
        groupby='month',
        # Use background tiles
        geo=True, crs=ccrs.Mercator(), tiles='CartoLight',
        title="Willow Flycatcher Migration (Americas)",
        xlim=(xmin, xmax), ylim=(ymin, ymax),
        frame_height=600,
        colorbar=False,
        widgets={'month': slider},
        widget_location='bottom',
        width=500,
        height=500
    )
)

# Save the plot
migration_plot.save('willow-flycatcher-migration.html', embed=True)

# Show the plot
migration_plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'ccae10cc-5026-4afa-801f-f8f6fdbdf3a4': {'version…

##### Migration Patterns

#### References

Afzal, P. (2024, January 4). *Endangered Willow Flycatchers in San Diego are adapting to climate change.* Cornell Lab of Ornithology. https://www.allaboutbirds.org/news/endangered-willow-flycatchers-in-san-diego-are-adapting-to-climate-change 

Bird Genoscape Project. (n.d.). *Willow Flycatcher.* https://www.birdgenoscape.org/willow-flycatcher 

Cornell Lab of Ornithology. (n.d.). *Willow Flycatcher life history.* All About Birds. https://www.allaboutbirds.org/guide/Willow_Flycatcher/lifehistory

National Audubon Society. (n.d.). *Willow Flycatcher.* Audubon. https://www.audubon.org/field-guide/bird/willow-flycatcher

Tahoe National Forest. (n.d.). *Willow Flycatcher - introduction.* U.S. Forest Service. https://www.fs.usda.gov/detail/tahoe/landmanagement/resourcemanagement/?cid=stelprdb5357314#:~:text=The%20scientific%20name%20for%20willow,t 





